In [1]:
import os
import numpy as np 
import cv2
import matplotlib.pyplot as plt
from omegaconf import OmegaConf

import torch
import torch.nn as nn

from Data.MVTEC_data import MVTecDataloader
from Models.STFPM import STFPM

In [2]:
args = OmegaConf.load('./Config/mvtec_STFPM_config.yaml')
os.makedirs(args.experiment_path, exist_ok=True)
os.makedirs(args.checkpoint_save, exist_ok=True)
np.random.seed(42)
torch.manual_seed(42)
args

{'experiment_name': 'STFPM', 'experiment_path': './Experiments/STFPM/mvtec/carpet', 'dataroot': './Datasets/mvtec/carpet', 'resume': False, 'checkpoint_save': './Experiments/STFPM/mvtec/carpet', 'checkpoint_load': './Experiments/STFPM/mvtec/carpet/best.pth.tar', 'log_path': './Experiments/STFPM/mvtec/carpet/carpet.log', 'start_epoch': 1, 'finish_epoch': 500, 'manualseed': 42, 'batchSize': 32, 'imageSize_h': 256, 'imageSize_w': 256, 'lossSize_h': 64, 'lossSize_w': 64, 'device': 'gpu', 'gpu_ids': 0, 'ngpu': 1, 'lr': 0.4, 'momentum': 0.9, 'weight_decay': 0.0001, 'sample_interval': 20}

In [3]:
train_loader, val_loader = MVTecDataloader(args.dataroot, batchSize=args.batchSize, imageSize_h=args.imageSize_h, imageSize_w=args.imageSize_w, is_train=True)
test_neg_loader, test_pos_loader = MVTecDataloader(args.dataroot, batchSize=args.batchSize, imageSize_h=args.imageSize_h, imageSize_w=args.imageSize_w, is_train=False)

In [4]:
Trainer = STFPM(args)

/home/gin/miniconda3/envs/vdt/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/gin/miniconda3/envs/vdt/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Checkpoint load fail.


/home/gin/miniconda3/envs/vdt/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [5]:
Trainer.train(train_loader, val_loader, test_neg_loader, test_pos_loader)

  0%|          | 0/500 [00:00<?, ?it/s]

[1/500] loss: 3.018981
Valid Loss: 2.0120394
AUC =  0.48836276083467095
Checkpoint saved successfully at epoch 1
[2/500] loss: 1.640610
Valid Loss: 2.1175640
AUC =  0.6577046548956661
Checkpoint saved successfully at epoch 2
[3/500] loss: 0.953990
Valid Loss: 0.6976763
AUC =  0.4791332263242376
[4/500] loss: 0.740139
Valid Loss: 0.0267853
AUC =  0.8836276083467095
Checkpoint saved successfully at epoch 4
[5/500] loss: 0.645670
Valid Loss: 0.0128897
AUC =  0.9478330658105939
Checkpoint saved successfully at epoch 5
[6/500] loss: 0.587208
Valid Loss: 0.0087758
AUC =  0.9622792937399679
Checkpoint saved successfully at epoch 6
[7/500] loss: 0.547799
Valid Loss: 0.0055536
AUC =  0.9775280898876404
Checkpoint saved successfully at epoch 7
[8/500] loss: 0.521615
Valid Loss: 0.0062946
AUC =  0.9153290529695024
[9/500] loss: 0.499048
Valid Loss: 0.0057467
AUC =  0.8940609951845907
[10/500] loss: 0.483732
Valid Loss: 0.0037382
AUC =  0.949438202247191
[11/500] loss: 0.467652
Valid Loss: 0.00370

KeyboardInterrupt: 

In [6]:
pos = Trainer.test(test_pos_loader)
neg = Trainer.test(test_neg_loader)

In [7]:
scores = []
for i in range(len(pos)):
    temp = cv2.resize(pos[i], (128, 192))
    scores.append(temp)
for i in range(len(neg)):
    temp = cv2.resize(neg[i], (128, 192))
    scores.append(temp)

scores = np.stack(scores)
gt_image = np.concatenate((np.ones(pos.shape[0], dtype=np.bool_), np.zeros(neg.shape[0], dtype=np.bool_)), 0)        

# auc_image_max = evaluate(gt_image, scores.max(-1).max(-1), metric='roc')
scores.max(-1).max(-1)

array([8.14376194e-02, 1.17080384e-02, 2.82905775e-03, 9.41141951e-03,
       3.29814935e-02, 6.25306356e-03, 2.10062583e-03, 1.91917494e-02,
       1.14655469e-01, 1.32854409e-02, 1.75816030e-02, 1.00623179e-01,
       6.63937312e-02, 7.22838156e-02, 1.10096682e-02, 3.06252986e-02,
       1.44852810e-02, 3.42285377e-03, 2.01595151e-02, 1.50847193e-01,
       3.67848640e-02, 4.35731979e-02, 6.29456528e-02, 1.80646563e-02,
       4.57492843e-02, 4.80912197e-02, 4.11236389e-02, 1.04348490e-02,
       5.00516947e-02, 2.23178156e-02, 6.98211454e-02, 1.05834045e-01,
       3.95228053e-02, 5.49225062e-02, 1.06132871e-01, 2.67208717e-02,
       3.00291036e-02, 6.41739718e-03, 1.84559647e-02, 2.61985511e-02,
       5.45366565e-02, 3.09775663e-02, 8.14575851e-02, 8.31486592e-02,
       1.48470655e-01, 5.84914684e-02, 5.61299575e-02, 6.32488253e-02,
       2.21372531e-02, 6.39471905e-02, 1.15872885e-01, 4.42887489e-02,
       2.35751639e-02, 6.58478471e-03, 5.40282566e-03, 2.14304086e-02,
      

In [10]:
from sklearn.metrics import roc_auc_score

In [11]:
roc_auc_score(gt_image, scores.max(-1).max(-1))

0.9361958266452649